In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".13"

In [2]:
import pandas as pd
from brain import BrainModel
from submodels import factories
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from random import shuffle

from lib.score import (
    fate_corr, score_both_size_norm, shrink_and_align_stats, score_stats_norm
)
from lib.preprocess import *
from lib.callback import (
    cell_number_callback, progenitor_number_callback, neuron_number_callback,
    TargetPopulation, TagNumberCallback,
)
from lib.sde.grn.grn4 import GRNMain4 as GRNMain
from lib.sde.mutate import mutate_grn4 as mutate_grn

from lib.ga.utils import weighted_selection_one, normalize_fitness_values
from jf.utils.export import Exporter
from jf.autocompute.jf import O, L
from itertools import product
import jf.models.stringmodel as sm
from lib.analyser import show_curve, show_curve_progenitor
import random
from lib.sde.mutate import mutate_gene4, mutate_tree

In [3]:
HISTORY = defaultdict(dict)
HALL_OF_FAME = []

In [4]:
_count = -1
def provide_id():
    global _count
    _count += 1
    return _count

In [5]:
REF = O(
    stats=pd.read_csv("reference/ref_tristate2.csv"),  # ref is a mean
)

In [6]:
def individual_generator(id_=-1, cb_init=None, nb_genes=4):
    return Solution(GRNMain(nb_genes, 0, 1, generate_funcs=cb_init), id_=id_)

In [7]:
_MUTATE_FUNC = mutate_grn

In [8]:
class Solution:
    def __init__(self, grn, id_=0, parent=-1):
        self.id = id_
        self.grn = grn
        self.parent = parent
        self.fit = -1
        self.stats = None
        
    def copy(self, id_=0):
        return Solution(self.grn.copy(), id_=id_, parent=self.id)
        
    def mutate(self):
        global _MUTATE_FUNC
        _MUTATE_FUNC(self.grn)

In [9]:
def score_bb_size(bb, ref, *args, **kwargs):
    s_prog = score_stats_norm(bb.stats, ref.stats, col_stats=f"progenitor_pop_size",
                     col_ref="progenitor_pop_size", norm=2.0, *args, **kwargs)
    s_all = score_stats_norm(bb.stats, ref.stats, col_stats=f"whole_pop_size",
                     col_ref="whole_pop_size", norm=2.0, *args, **kwargs)
    s_neuron = score_stats_norm(bb.stats, ref.stats, col_stats=f"neuron_pop_size",
                     col_ref="neuron_pop_size", col_norm_data="progenitor_pop_size",
                       col_norm_ref="progenitor_pop_size", norm=2.0, *args, **kwargs)

    return np.mean(s_prog + s_all + s_neuron)

In [10]:
def setup_tag(cp):
    indexes = list(cp.base_population.keys())
    shuffle(indexes)
    splits = np.array_split(indexes, 3)
    for i, ls in enumerate(splits):
        for idx in ls:
            cp.base_population[idx].tag["subbrain"] = i

In [11]:
def get_bb(prun, grn):
    ccls = factories["grn4"](grn=grn)
    callbacks = dict(
        progenitor_pop_size=progenitor_number_callback,
        whole_pop_size=cell_number_callback,
        neuron_pop_size=neuron_number_callback,
    )
    bb = BrainModel(time_step=0.5, verbose=False, start_population=prun.size, max_pop_size=3e2,
            cell_cls=ccls, end_time=prun.end_time, start_time=56, silent=True, opti=True,
              run_tissue=True, monitor_callbacks=callbacks, tag_func=setup_tag)
    return bb

In [12]:
def run_grn(prun, grn):
    get_bb(prun, grn)
    bb.run()
    return bb

In [13]:
def fitness_multistep(prun, grn, steps):
    total_fitness = 0
    stop = False
    previous_time = None
    bb = get_bb(prun, grn)
    # first step
    for step in steps:
        if not bb.run_until(step.end_time):
            stop = True
        # score_step = score_both_size(bb.stats, prun.ref, max_step=step.end_time, min_step=previous_time)
        score_step = step.score_func(bb, prun.ref, max_step=step.end_time, min_step=step.start_time)
        fitness_step = 1.0 / score_step
        fitness_step = min(fitness_step, step.max_fitness)
        total_fitness += fitness_step
        if fitness_step < step.min_fitness or stop:
            return total_fitness, bb.stats
        else:
            previous_time = step.end_time
            step.passed()
        
    return total_fitness, bb.stats

In [14]:
def mean_sd_fitness(prun, grn, run=3):
    fitnesses = [fitness_multistep(prun, grn) for i in range(run)]
    return np.mean(fitnesses), np.std(fitnesses)

In [15]:
def multi_fitness(*args):
    fitnesses = [fitness_multistep(*args) for i in range(3)]
    scores = [x[0] for x in fitnesses]
    return fitnesses[scores.index(min(scores))]

In [16]:
def do_init(prun):
    return individual_generator(provide_id(), prun.cb_init)

def do_fitness(prun, sol):
    fitness, stats = fitness_multistep(prun, sol.grn, prun.steps)
    return fitness, stats

def do_selection(prun, pop_fit, pop):
    if len(pop) < prun.min_pop:
        return individual_generator(provide_id(), prun.cb_init)
    
    pop_fit = normalize_fitness_values(pop_fit)
    
    return weighted_selection_one(pop, pop_fit, lambda x: individual_generator(x, prun.cb_init),
                                  new_fitness=0.5, id_=provide_id())[0]

def do_mutation(prun, sol):
    sol.mutate()
    return sol

In [17]:
class ObjectiveStep(O):
    start_time = 0
    end_time = 0
    max_fitness = 3
    min_fitness = 1
    name = ""
    _passed = False
    
    def reset(self):
        self._passed = False
    
    def passed(self):
        if self._passed:
            return
        print(f"Step {self.name} passed !")
        self._passed = True
    
example_steps = [
    ObjectiveStep(name="The ONE", start_time=56, end_time=86, score_func=score_bb_size, min_fitness=0.2),
]

class ParamRun(O):
    pop_size = 10
    n_gen = 30
    current_gen = 0
    end_time = 86
    ref = REF
    min_pop = 25
    max_pop = 50

def get_prun(size=5, exponent=1):
    prun = ParamRun()
    prun.cb_init = dict()
    prun.size = size
    prun.exponent = exponent
    prun.steps = example_steps
    return prun

In [18]:
def main(prun):
    prun.history = defaultdict(dict)
    # exporter = Exporter()
    best = 0
    sol = do_init(prun)
    pop = [sol]
    for generation in range(prun.n_gen * prun.pop_size):
        # args.generation = generation
        # objective.new_trial()
        fit, stats = do_fitness(prun, sol)
        sol.fit = fit
        sol.stats = stats
        
        # history
        # print(f"Fitness = {fit}", end="\t\t")
        if generation % 100 == 0:
            print(f"Step {generation}")
        if fit > best:
            print(f"++ Best {fit} for generation {generation}")
            best = fit
            
        monitor = sol
        prun.history[generation] = monitor
        # exporter(monitor, f"generation_g{generation}")
        
        # TODO get the stats associated with the best scores
        sub_pop = pop[-prun.max_pop:]
        sol = do_selection(prun, [s.fit for s in sub_pop], sub_pop)
            
        sol = do_mutation(prun, sol)
        pop.append(sol)
        
    print("OVER")
    return best

In [19]:
prun = get_prun()
# res = main(prun)

In [20]:
def get_score_parametrized(cb_init, name=None):
    args = get_prun()
    args.cb_init = cb_init
    args.name = name
    main(args)
    p1 = L(args.history.values()).fit
    idx = p1.index(max(p1))
    sol = args.history[idx]
    scores = [fitness_multistep(args, sol.grn, args.steps)[0] for i in range(10)]
    print("Final score", np.mean(scores), "+-", np.std(scores))
    return np.mean(scores), np.std(scores), max(p1)

In [21]:
def mutate_grn_verysparse(grn, temperature=0.1):
    grn.set_mutable()
    shape = grn._params.shape
    r = random.random()
    param_prob = 0.8
    if r < param_prob:
        mask = (np.random.uniform(0, 1, shape) > 0.9)
        coeff = np.random.normal(0, temperature, shape)
        true_coeff = mask * coeff + 1
        grn._params *= true_coeff
    else:
        one_gene = random.choice(grn.genes)
        one_gene.tree = mutate_tree(one_gene.tree, one_gene.get_labels_not_in_tree())
    grn.compile()

In [22]:
model = sm.StringModel("expfit_func{mutfunc}_i{i}")
mut_func = dict(
    fullctrl=lambda x: mutate_grn_verysparse(x),
    full005=lambda x: mutate_grn_verysparse(x, temperature=0.05),
    full02=lambda x: mutate_grn_verysparse(x, temperature=0.2),
)

In [23]:
callback_init = dict(
    init=lambda: np.random.beta(1.5, 3) * 3,
    b=lambda: np.random.beta(1.5, 3) * 5,
    expr=lambda: 1,
    deg=lambda: 0.1,
)

In [24]:
# main loop
exporter = Exporter(name="exp_mutfunc5_280222")
for (funcname, func), i in product(mut_func.items(), range(10)):
    name = model.fill(mutfunc=funcname, i=i)
    print(name)
    if name in exporter.list():
        continue
    _MUTATE_FUNC = func
    res = get_score_parametrized(callback_init)
    exporter(res, name)

Exporting at output/exp_mutfunc5_280222
expfit_funcfullctrl_i0
Step 0
++ Best 0.0062184478395978906 for generation 0
++ Best 0.008417107707737508 for generation 13
++ Best 0.011242719709898883 for generation 52
++ Best 0.018295055144864434 for generation 66
++ Best 0.022711430840175618 for generation 94
Step 100
Step 200
OVER
Final score 0.013236953956730899 +- 0.003485354169011998
expfit_funcfullctrl_i1
Step 0
++ Best 0.006011546907708517 for generation 0
++ Best 0.009873666434729342 for generation 1
++ Best 0.011827873284505082 for generation 4
++ Best 0.012063431781002274 for generation 18
Step 100
Step 200
++ Best 0.01709869984563481 for generation 271
OVER
Final score 0.016217107947353877 +- 0.0034015625929216193
expfit_funcfullctrl_i2
Step 0
++ Best 0.0027039176097328 for generation 0
++ Best 0.0035347639087328422 for generation 1
++ Best 0.005711397725844478 for generation 2
++ Best 0.005716838582739354 for generation 4
++ Best 0.005717054289391078 for generation 5
++ Best 0.005

In [25]:
dicts = [dict(**model.extract(x), **dict(zip(("mean", "sd", "max"), exporter.load(x)))) for x in exporter.list() if model.match(x)]

In [26]:
df = pd.DataFrame(dicts)

In [27]:
df.head()

,mutfunc,i,mean,sd,max
0,full02,0,0.018073,0.013398,0.051523
1,full005,7,0.021720,0.004648,0.040424
2,full02,9,0.036315,0.008987,0.028303
3,full005,3,0.015204,0.011172,0.032127
4,fullctrl,7,0.041481,0.018999,0.078194


In [28]:
new_df = df.groupby(["mutfunc"]).mean()
new_df

,mean,sd,max
mutfunc,,,
full005,0.017957,0.007204,0.031487
full02,0.022254,0.007600,0.033680
fullctrl,0.023740,0.010107,0.040516


In [29]:
for i, row in new_df.iterrows():
    print(row.name, row["mean"], row["mean"] - row.sd / np.sqrt(10), row["mean"] + row.sd / np.sqrt(10))

full005 0.017957347256524238 0.015679202812287987 0.02023549170076049
full02 0.02225386027416312 0.019850537109706037 0.024657183438620202
fullctrl 0.023740299690169386 0.020544054475227544 0.02693654490511123


In [30]:
row.name

'fullctrl'